In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import shap
import os
import matplotlib.pyplot as plt

In [ ]:
cwd = os.getcwd()
print("Current Working Directory:", cwd)

In [ ]:
import avocado

classifier = avocado.load_classifier('redshift_weight')
features_path = "plasticc_augment"
dataset = avocado.load(features_path, metadata_only=True)
dataset.load_raw_features()

In [ ]:
metadata = dataset.metadata
data = dataset.load_raw_features()
data = data.dropna(axis=1)

In [ ]:
model_1 = classifier.classifiers[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

Y = metadata['class']
Y_encoded = le.fit_transform(Y)

In [ ]:
class SelectiveFeaturizer(avocado.plasticc.PlasticcFeaturizer):
    def select_features(self, raw_features):
        return pd.read_pickle("experiments/reduced_data.pkl")

class GenerativeFeaturizer(avocado.plasticc.PlasticcFeaturizer):
    def select_features(self, raw_features):
        # generating the features avocado found to be best
        original_features = super().select_features(raw_features)

        # Adding aditional features
        cleaned = raw_features.dropna(axis=1)
        cleaned = cleaned.drop(['ra', 'decl', 'mwebv', 'ddf', 'count'], axis=1)
        
        extra_features = self.select_another_set_features(cleaned)

        # Creating a unified dataframe
        combined = pd.concat([original_features, extra_features], axis=1)
        combined = combined.loc[:, ~combined.columns.duplicated()]
        
        return combined
    
    from sklearn.base import BaseEstimator, TransformerMixin
    
    class PairwiseCombinations(BaseEstimator, TransformerMixin):
        
        def __init__(self, operations=["add", "sub", "mul", "div"]):
            self.operations = operations

        def fit(self, X, y=None):
            self.features_ = X.select_dtypes(include=np.number).columns.tolist()
            return self

        def transform(self, X):
            from itertools import combinations
            
            X = X.copy()
            new_features = []

            # Get combinations of numeric columns
            pairs = list(combinations(self.features_, 2))

            for col1, col2 in pairs:
                idx = X.index  # Ensure all new Series use same index
                if "add" in self.operations:
                    new_features.append(pd.Series(X[col1] + X[col2], name=f"{col1}_plus_{col2}", index=idx))
                if "sub" in self.operations:
                    new_features.append(pd.Series(X[col1] - X[col2], name=f"{col1}_minus_{col2}", index=idx))
                if "mul" in self.operations:
                    new_features.append(pd.Series(X[col1] * X[col2], name=f"{col1}_mul_{col2}", index=idx))
                if "div" in self.operations:
                    with np.errstate(divide='ignore', invalid='ignore'):
                        new_features.append(pd.Series(X[col1] / X[col2].replace(0, np.nan), name=f"{col1}_div_{col2}", index=idx))

            # Combine all new columns at once
            if new_features:
                new_df = pd.concat(new_features, axis=1)
                return pd.concat([X, new_df], axis=1)

            return X  # fallback: return original if nothing new

    def select_another_set_features(self, raw_features):
        transformer = self.PairwiseCombinations(operations=['div'])
        return transformer.fit_transform(raw_features).dropna(axis=1)

dataset = avocado.load(features_path, metadata_only=True)
dataset.load_raw_features()
    
classifier = avocado.LightGBMClassifier(
        'test',
        SelectiveFeaturizer(),
        class_weights=None,
        weighting_function=avocado.evaluate_weights_redshift,
)
classifier.train(dataset)

In [ ]:
X1 = dataset.select_features(SelectiveFeaturizer())
model_1 = classifier.classifiers[0]
explainer_1 = shap.TreeExplainer(model_1)
shap_1 = explainer_1.shap_values(X1)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

corr = X1.corr(method='pearson')
corr = 1 - corr.abs()

clustering = AgglomerativeClustering(
    affinity='precomputed',
    linkage='complete',
    distance_threshold=0.3,
    n_clusters=None
)
labels = clustering.fit_predict(corr)
np.unique(labels).shape[0]

In [ ]:
from collections import defaultdict

# This dict will store selected features per class
class_to_top_features = defaultdict(set)

cluster_id_to_features = {}
for col, cluster_id in zip(X1.columns, labels):
    cluster_id_to_features.setdefault(cluster_id, []).append(col)

# Loop over each class
for class_id in np.unique(Y):
    mask = (Y == class_id)
    
    # Get SHAP values for this class
    shap_values_class = shap_1[class_map[class_id]][mask]
    this_df = pd.DataFrame(shap_values_class, columns=X1.columns)

    # Compute median SHAP per feature
    median_shap = this_df.median().abs()

    # Aggregate by cluster
    cluster_medians = {}
    for cluster_id, features in cluster_id_to_features.items():
        valid_features = [f for f in features if f in median_shap.index]
        if valid_features:
            cluster_medians[cluster_id] = median_shap[valid_features].mean()

    # Sort clusters by importance
    cluster_shap_series = pd.Series(cluster_medians).sort_values(ascending=False)
    normalized_shap = cluster_shap_series / cluster_shap_series.sum()
    cumulative_shap = normalized_shap.cumsum()

    # Get top clusters contributing to 99% SHAP
    top_clusters = cumulative_shap[cumulative_shap <= 0.99].index
    if len(cumulative_shap) > len(top_clusters):
        top_clusters = cumulative_shap[cumulative_shap <= 0.99].append(
            cumulative_shap[cumulative_shap > 0.99].iloc[:1]
        ).index

    # Add features from top clusters
    for cluster in top_clusters:
        class_to_top_features[class_id].update(cluster_id_to_features[cluster])

# Union of all top features across classes
all_top_features = set().union(*class_to_top_features.values())

final_top_features = set()

for class_id in np.unique(Y):
    mask = (Y == class_id)
    
    # Get SHAP values and median SHAP
    shap_values_class = shap_1[class_map[class_id]][mask]
    this_df = pd.DataFrame(shap_values_class, columns=X1.columns)
    median_shap = this_df.median().abs()
    
    for cluster_id, features in cluster_id_to_features.items():
        # Check if any feature from this cluster is in the selected set for this class
        if any(f in class_to_top_features[class_id] for f in features):
            valid_features = [f for f in features if f in median_shap.index]
            if valid_features:
                # Add top features to the united set
                top_k = median_shap[valid_features].sort_values(ascending=False).head(1).index
                final_top_features.update(top_k)

In [ ]:
X2 = dataset.select_features(avocado.plasticc.PlasticcFeaturizer())

common_columns = list(set(X2.columns) & set(X1[final_top_features].columns))
not_in_common = list(set(X2.columns) - set(X1[final_top_features].columns))

X2[not_in_common]

In [ ]:
test = X1
test.to_pickle("experiments/reduced_data.pkl")
test